[![AnalyticsDojo](https://github.com/rpi-techfundamentals/fall2018-materials/blob/master/fig/final-logo.png?raw=1)](http://rpi.analyticsdojo.com)
<center><h1>Fastai - Shelter Animals Outcomes</h1></center>
<center><h3><a href = 'http://rpi.analyticsdojo.com'>rpi.analyticsdojo.com</a></h3></center>


In [1]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

Looking in links: https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html


In [2]:
from fastai import *
from fastai.tabular import * 
import numpy as np
import pandas as pd
import pandas as pd

train= pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')

In [3]:
import datetime
def transform_date(df):
    date = pd.to_datetime(df.DateTime)
    df['Hour'] = date.dt.hour
    df['WeekDay'] = date.dt.weekday
    df['Month'] = date.dt.month

transform_date(train)
transform_date(test)

In [4]:
#Create a categorical variable from the family count 
def name(x):
    if x == 'Unnamed':
        return 'Unnamed'
    else:
        return 'Named'
    
def clean_sex(x):
    if 'Female' in x:
        return 'Female'
    elif 'Male' in x:
        return 'Male' 
    else:
        return 'Unknown'
    
def clean_fixed(x):
    if 'Spay' in x:
        return 'Fixed'
    elif 'Neuter' in x:
        return 'Fixed'
    elif 'Intact' in x:
        return 'Intact'
    else:
        return 'Unknown'
    
def scale_ages(x):
    if 'year' in x:
        return int(x.split(' ')[0])*365
    elif 'month' in x:
        return int(x.split(' ')[0])*12
    elif 'week' in x:
        return int(x.split(' ')[0])*7
    elif 'day' in x:
        return int(x.split(' ')[0])
    else:
        1
        
def category_age(x):
    if x <= 365: #less than or equal to 1 year
        return 'Baby'
    elif 365 < x <= 1095: #greater than 1 year or equal to 3 years
        return 'Young adult' 
    elif 1095 < x <= 1825: #greater than 3 years or equal to 5 years
        return 'Adult'
    else: # greater than 5 years
        return 'Senior'
        
def category_season(x):
    if x == 12: #December
        return 'Winter'
    elif 1<= x <= 2: #January, Feburary
        return 'Winter'
    elif 3 <= x <= 5: #March, April, May
        return 'Spring'
    elif 6 <= x <= 8: #June, July, August
        return 'Summer'
    else: #September, October, November
        return 'Fall'

for df in [train, test]:
    df['Name'] = df['Name'].fillna('Unnamed')
    df['NameCategory'] = df['Name'].apply(name)
    df['SexuponOutcome'] = df['SexuponOutcome'].fillna('Unknown')
    df['Sex'] = df['SexuponOutcome'].apply(clean_sex)
    df['FixedIntact'] = df['SexuponOutcome'].apply(clean_fixed)
    df['AgeuponOutcome']=df['AgeuponOutcome'].fillna('1 year')
    df['AgeinDays'] = df['AgeuponOutcome'].apply(scale_ages)
    df['AgeCategory'] = df['AgeinDays'].apply(category_age)
    df['Season'] = df['Month'].apply(category_season)
    df['Breed'] = df['Breed'].fillna('Unknown')
    df['IsMix'] = df['Breed'].apply(lambda x: 'Mixed' if 'Mix' in x else 'Pure')
    df['IsMultipleBreeds'] = df['Breed'].apply(lambda x: 'Multiple' if '/' in x else 'Pure')
    df['IsMultipleColors'] = df['Color'].apply(lambda x: 'Mutiple' if '/' in x else 'Pure')
    
def clean_breed(df):
    breeds = df['Breed'].apply(lambda x: x.split('/'))
    df['Breed1'] = breeds.apply(lambda x: x[0].replace(' Mix', ''))
    df['Breed2'] = breeds.apply(lambda x: x[1] if len(x)>1 else x[0].replace(' Mix', ''))  
clean_breed(train)
clean_breed(test)

def clean_color(df):
    colors = train['Color'].apply(lambda x: x.split('/'))
    df['Color1'] = colors.apply(lambda x: x[0])
    df['Color2'] = colors.apply(lambda x: x[1] if len(x)>1 else x[0])
clean_color(train)
clean_color(test)
    
test.isnull().sum(axis = 0)

ID                  0
Name                0
DateTime            0
AnimalType          0
SexuponOutcome      0
AgeuponOutcome      0
Breed               0
Color               0
Hour                0
WeekDay             0
Month               0
NameCategory        0
Sex                 0
FixedIntact         0
AgeinDays           0
AgeCategory         0
Season              0
IsMix               0
IsMultipleBreeds    0
IsMultipleColors    0
Breed1              0
Breed2              0
Color1              0
Color2              0
dtype: int64

In [5]:
train.head()
# train.describe()

,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,...,AgeinDays,AgeCategory,Season,IsMix,IsMultipleBreeds,IsMultipleColors,Breed1,Breed2,Color1,Color2
0,A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White,...,365,Baby,Winter,Mixed,Pure,Mutiple,Shetland Sheepdog,Shetland Sheepdog,Brown,White
1,A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby,...,365,Baby,Fall,Mixed,Pure,Pure,Domestic Shorthair,Domestic Shorthair,Cream Tabby,Cream Tabby
2,A686464,Pearce,2015-01-31 12:28:00,Adoption,Foster,Dog,Neutered Male,2 years,Pit Bull Mix,Blue/White,...,730,Young adult,Winter,Mixed,Pure,Mutiple,Pit Bull,Pit Bull,Blue,White
3,A683430,Unnamed,2014-07-11 19:09:00,Transfer,Partner,Cat,Intact Male,3 weeks,Domestic Shorthair Mix,Blue Cream,...,21,Baby,Summer,Mixed,Pure,Pure,Domestic Shorthair,Domestic Shorthair,Blue Cream,Blue Cream
4,A667013,Unnamed,2013-11-15 12:52:00,Transfer,Partner,Dog,Neutered Male,2 years,Lhasa Apso/Miniature Poodle,Tan,...,730,Young adult,Fall,Pure,Multiple,Pure,Lhasa Apso,Miniature Poodle,Tan,Tan


In [8]:
dep_var = 'OutcomeType'
cat_names = ['AnimalType', 'NameCategory', 'Sex', 'AgeCategory', 'FixedIntact', 'Season', 'IsMix', 'IsMultipleBreeds', 'Breed1', 'Breed2', 'IsMultipleColors', 'Color1', 'Color2']
cont_names = ['AgeinDays']
procs = [FillMissing, Categorify, Normalize]
test_data = (TabularList.from_df(test, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs))

In [9]:
data = (TabularList.from_df(train, path='.', cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(0,200)))
                           .label_from_df(cols=dep_var)
                           .add_test(test_data, label=0)
                           .databunch())

Exception: Your validation data contains a label that isn't present in the training set, please fix your data.

In [ ]:
#Shows the Data
data.show_batch()

In [ ]:
#Define our Learner
learn = tabular_learner(data, layers=[300,100], metrics=accuracy)
#fit the learner
learn.fit(7, 1e-2)  #Number of epocs and the learning rate. learn.save('final_train')

In [ ]:
#Show the results
learn.show_results()

In [ ]:
#This will get predictions
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = to_np(np.argmax(predictions, 1))
print(labels)
labels.shape

In [ ]:
#Writing to File
submission=pd.DataFrame(test.loc[:,['ID']])
submission['OutcomeType']=labels
# submission['OutcomeTypeName'] = learn['target']
#Any files you save will be available in the output tab below

submission.to_csv('FastAIsubmission2.csv', index=False)

In [ ]:
# from google.colab import files
# files.download('submission.csv')